In [2]:
# import python libraries
import os
# from pprint import pprint
# import matplotlib.pyplot as plt
# import xarray as xr
# import rioxarray
# import geopandas as gpd
# import pandas as pd

# import hydromt and setup logging
import hydromt
from hydromt.log import setuplog

In [3]:
fiat_logger = setuplog("prepare data catalog", log_level=10)
fiat_path   = "C:\\Users\\fuentesm\\CISNE\\HydroMT_sprint_sessions\\Model_Builder"
fiat_crs        = 4326
fiat_layers =["Current_prob_event_set_combined_doNothing_withSeaWall_RP=1_max_flood_depth","Current_prob_event_set_combined_doNothing_withSeaWall_RP=2_max_flood_depth"]

2023-03-08 17:20:32,043 - prepare data catalog - log - INFO - HydroMT version: 0.7.0


In [4]:
def write_yml(fiat_path:str,crs:str):
    yml_str = f"""
    meta:
      root: {fiat_path}

    merit_hydro_local:
      data_type: RasterDataset
      driver: raster
      kwargs:
        chunks:
          x: 6000
          y: 6000
      meta:
        category: topography
      rename:
        hnd: height_above_nearest_drain
      path: merit_hydro/*.tif

    max_depth:
      path: Hazard/kingTide_SLR_max_flood_depth.tif
      data_type: RasterDataset
      driver: raster
      crs: {crs}
      meta:
        category: flood 

    flood_maps:
      path: Hazard/*.tif
      data_type: RasterDataset
      driver: raster
      crs: {crs}
      meta:
        category: flood 

    flood_maps_varaible:
      path: Hazard/{{variable}}.tif
      data_type: RasterDataset
      driver: raster
      crs: {crs}
      meta:
        category: flood

    vito_local:
      path: vito.tif
      data_type: RasterDataset
      driver: raster
      crs: {crs}
      meta:
          category: landuse

    exposure:
        path: NSI_Charleston/charleston_test_dataset_EPGS4326.shp
        data_type: GeoDataFrame
        driver: vector
        crs: {crs}
        meta:
          category: expusure

    vulnerability:
        path: Vulnerability/AllDDF.xlsx
        data_type: DataFrame
        driver: xlsx
        meta:
          category: vulnerability

    social_vulnerability:
        path: Social_Vulnerability/
        data_type: DataFrame
        driver: xlsx
        meta:
          category: social_vulnerability

    """
    # overwrite data catalog
    data_lib = fiat_path + "/fiat_catalog.yml"

    with open(data_lib, mode="w") as f:
        f.write(yml_str)
    return data_lib


def read_raster(data_lib:str,logger,raster_name:str, layers):
    #data_catalog = hydromt.DataCatalog(data_libs=["artifact_data=v0.0.6"])
    data_catalog = hydromt.DataCatalog(data_libs=[data_lib], logger=logger)
    print(data_catalog.sources.keys())
    da = data_catalog.get_rasterdataset(raster_name, variables=layers)
    return da

In [9]:
fiat_data_lib        = write_yml(fiat_path, fiat_crs)
flood_maps           = read_raster(fiat_data_lib,fiat_logger, "flood_maps_varaible", None)
exposure             = read_raster(fiat_data_lib,fiat_logger, "exposure", None)
vulnerability        = read_raster(fiat_data_lib,fiat_logger, "vulnerability", None)
#social_vulnerability = read_raster(fiat_data_lib,fiat_logger, "social_vulnerability", None)

2023-03-08 17:24:20,752 - prepare data catalog - data_catalog - INFO - Parsing data catalog from C:\Users\fuentesm\CISNE\HydroMT_sprint_sessions\Model_Builder/fiat_catalog.yml
dict_keys(['merit_hydro_local', 'max_depth', 'flood_maps', 'flood_maps_varaible', 'vito_local', 'exposure', 'vulnerability', 'social_vulnerability'])
2023-03-08 17:24:20,766 - prepare data catalog - data_catalog - INFO - DataCatalog: Getting flood_maps_varaible RasterDataset raster data from C:\Users\fuentesm\CISNE\HydroMT_sprint_sessions\Model_Builder\Hazard\{variable}.tif
2023-03-08 17:24:21,033 - prepare data catalog - data_catalog - INFO - Parsing data catalog from C:\Users\fuentesm\CISNE\HydroMT_sprint_sessions\Model_Builder/fiat_catalog.yml
dict_keys(['merit_hydro_local', 'max_depth', 'flood_maps', 'flood_maps_varaible', 'vito_local', 'exposure', 'vulnerability', 'social_vulnerability'])
2023-03-08 17:24:21,046 - prepare data catalog - data_catalog - INFO - DataCatalog: Getting exposure RasterDataset vector

In [10]:
print(type(flood_maps))
print(type(exposure))
print(type(vulnerability))
#print(type(social_vulnerability))

<class 'xarray.core.dataset.Dataset'>
<class 'geopandas.geodataframe.GeoDataFrame'>
<class 'pandas.core.frame.DataFrame'>


In [11]:
flood_maps

<xarray.Dataset>
Dimensions:                                                                       (
                                                                                   x: 5770,
                                                                                   y: 7272)
Coordinates:
  * x                                                                             (x) float64 ...
  * y                                                                             (y) float64 ...
    spatial_ref                                                                   int32 ...
Data variables:
    Current_prob_event_set_combined_doNothing_withSeaWall_RP=5_max_flood_depth    (y, x) float32 dask.array<chunksize=(7272, 5770), meta=np.ndarray>
    kingTide_SLR_max_flood_depth                                                  (y, x) float32 dask.array<chunksize=(7272, 5770), meta=np.ndarray>
    Current_prob_event_set_combined_doNothing_withSeaWall_RP=100_max_flood_depth  (y, x) float32 dask.array<chunksize=(7272, 5770), meta=np.ndarray>
    Current_prob_event_set_combined_doNothing_withSeaWall_RP=10_max_flood_depth   (y, x) float32 dask.array<chunksize=(7272, 5770), meta=np.ndarray>
    Current_prob_event_set_combined_doNothing_withSeaWall_RP=50_max_flood_depth   (y, x) float32 dask.array<chunksize=(7272, 5770), meta=np.ndarray>
    Current_prob_event_set_combined_doNothing_withSeaWall_RP=25_max_flood_depth   (y, x) float32 dask.array<chunksize=(7272, 5770), meta=np.ndarray>
    Current_prob_event_set_combined_doNothing_withSeaWall_RP=2_max_flood_depth    (y, x) float32 dask.array<chunksize=(7272, 5770), meta=np.ndarray>
    Current_prob_event_set_combined_doNothing_withSeaWall_RP=1_max_flood_depth    (y, x) float32 dask.array<chunksize=(7272, 5770), meta=np.ndarray>
Attributes:
    category:  flood

In [12]:
from hydromt_fiat.workflows.hazard import Hazard

In [16]:
map_fn   = "C:/Users/fuentesm/CISNE/HydroMT_sprint_sessions/Model_Builder/Hazard/kingTide_SLR_max_flood_depth.tif"    				
map_type = "water_depth"								
rp       = 2              								
crs      = 3452									
nodata   = -9999									
var      = None									
chunks   = 100									


In [17]:
test_hazard = Hazard.setup_hazard(map_fn=map_fn, map_type=map_type, rp=rp, crs=crs,nodata=nodata,var=None,chuncks=chunks)

TypeError: Hazard.setup_hazard() missing 1 required positional argument: 'self'